# IEEE Camera Model Identification
## Library imports

In [1]:
import os
from multiprocessing import Pool, cpu_count
import itertools

import numpy as np
import pandas as pd

from skimage.data import imread
from skimage.transform import resize
from skimage.exposure import adjust_gamma

from sklearn.utils import class_weight

from io import BytesIO
from PIL import Image
import urllib
from scipy import ndimage

from random import sample

import matplotlib.pyplot as plt
%matplotlib inline

## Keras
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.optimizers import Adam, SGD, Adadelta, Nadam
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import Sequence
from keras import backend as K

from keras.applications.densenet import DenseNet201
import math

## MultiGPU Code
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))
import keras.utils.training_utils
from keras.utils import multi_gpu_model
gdev = keras.utils.training_utils._get_available_devices()
gdev_count = 0
for i, n in enumerate(gdev):
    if 'device:GPU' in n:
        gdev_count+=1
if gdev_count > 0:
    print('Found {} GPUs, will attempt to use all of them.'.format(gdev_count))
else:
    gdev_count=1
    print('Did not find any GPUs, this will be SLOW!')

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 8 GPUs, will attempt to use all of them.


## Set up paths and other variables

In [2]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "session_name = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [3]:
input_path = '/media/share/data/kaggle/ieee-camera/'
train_path = os.path.join(input_path,'train')
test_path = os.path.join(input_path,'test')
flickr_path = os.path.join(input_path,'flickr_images')
img_rows = 512
img_cols = 512

In [4]:
cameras = os.listdir(train_path)

train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(os.path.join(train_path,camera))):
        train_images.append((camera, fname))

train = pd.DataFrame(train_images, columns=['camera', 'fname'])
print(train.shape)
train.head(5)

(2750, 2)


,camera,fname
0,Motorola-X,(MotoX)1.jpg
1,Motorola-X,(MotoX)10.jpg
2,Motorola-X,(MotoX)100.jpg
3,Motorola-X,(MotoX)101.jpg
4,Motorola-X,(MotoX)102.jpg


In [5]:
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)

test = pd.DataFrame(test_images, columns=['fname'])
print(test.shape)
test.head(5)

(2640, 1)


,fname
0,img_0002a04_manip.tif
1,img_001e31c_unalt.tif
2,img_00275cf_manip.tif
3,img_0034113_unalt.tif
4,img_00344b7_unalt.tif


In [6]:
flickr_images = []
for camera in cameras:
    for fname in sorted(os.listdir(os.path.join(flickr_path,camera))):
        if fname.endswith('jpg') or fname.endswith('png'):
            flickr_images.append((camera, fname))

flickr = pd.DataFrame(flickr_images, columns=['camera', 'fname'])
print(flickr.shape)
flickr.head(5)

(11603, 2)


,camera,fname
0,Motorola-X,23936849344_51b33456f3_o.jpg
1,Motorola-X,23954069334_58f4e3a6d7_o.jpg
2,Motorola-X,23956920324_19d0cdee07_o.jpg
3,Motorola-X,23957852063_2194f748d2_o.jpg
4,Motorola-X,23962520803_5c85ac0b75_o.jpg


In [7]:
# Compute a class_weight
# orig_class_weights = class_weight.compute_class_weight('balanced', np.asarray(cameras), train['camera'])
# print(orig_class_weights)
flickr_class_weights = class_weight.compute_class_weight('balanced', np.asarray(cameras), flickr['camera'])
print(flickr_class_weights)

[0.71535142 1.45219024 0.90719312 0.79910468 0.96691667 1.73179104
 0.89391371 1.65757143 0.90366044 0.89322556]


In [8]:
np.array([[0,0,0,1],[0,1,0,0]]).argsort(axis=1)[:,::-1][:,0]

array([3, 1])

## Define a function to build image set using the training data set
This way we can apply multiprocessing to the function

In [9]:
hpf = np.array([[-1,2,-2,2,-1],
               [2,-6,8,-6,2],
               [-2,8,-12,8,-2],
               [2,-6,8,-6,2],
               [-1,2,-2,2,-1]], dtype=np.float32)
hpf = (1/12)*hpf
hpf = np.dstack([np.zeros(hpf.shape),hpf,np.zeros(hpf.shape)])

In [10]:
def build_train_image_set(j):
    # Read image
    img = np.array(Image.open(os.path.join(os.path.join(train_path,train['camera'].iloc[j]),train['fname'].iloc[j])), dtype=np.uint8)
    
    # Add some augmentation
    #JPEG compression with quality factor = 70
    #JPEG compression with quality factor = 90
    #resizing (via bicubic interpolation) by a factor of 0.5
    #resizing (via bicubic interpolation) by a factor of 0.8
    #resizing (via bicubic interpolation) by a factor of 1.5
    #resizing (via bicubic interpolation) by a factor of 2.0
    #gamma correction using gamma = 0.8
    #gamma correction using gamma = 1.2
    if np.random.randint(2) < 1:
        tmp_stream = BytesIO()
        tmp_img = Image.fromarray(img)
        tmp_img.save(tmp_stream, format='jpeg', quality=int(np.random.choice([70,90])))
        tmp_stream.seek(0)
        img = np.array(Image.open(tmp_stream))
        img = np.array(resize(img,np.round(np.multiply(img.shape[:2],np.random.choice([0.5,0.8,1.5,2.0]))),
                              mode='reflect', preserve_range=True, order=3), dtype=np.uint8)
        img = adjust_gamma(img, gamma=np.random.choice([0.8,1.2]))
    
    # Get dimensions
    dim_x = img.shape[0]
    dim_y = img.shape[1]
    
    # Set temp img
    tmp_img = ndimage.convolve(img.astype(np.float32), hpf, mode='mirror')
    tmp_img = img.astype(np.float32) - tmp_img
    tmp_img = tmp_img.astype(np.float16)
    
    # Find out if image needs padding using %
#     if (dim_x % img_rows) != 0:
#         tmp_img = np.pad(tmp_img, ([0,img_rows-(dim_x % img_rows)],[0,0],[0,0]), 'reflect')
#     if (dim_y % img_cols) != 0:
#         tmp_img = np.pad(tmp_img, ([0,0],[0,img_cols-(dim_y % img_cols)],[0,0]), 'reflect')
    
    # Next, extract a random square
    cropped_imgs = []
    labels = []
    for i in range(25):
        m = np.random.randint(math.floor(dim_x / img_rows))
        n = np.random.randint(math.floor(dim_y / img_cols))
        xx_img = tmp_img[m*img_rows:(m+1)*img_rows,n*img_cols:(n+1)*img_cols,:]
        lab = np.zeros(len(cameras))
        lab[cameras.index(train['camera'].iloc[j])] = 1
        cropped_imgs += [xx_img]
        labels += [lab]
    
    return cropped_imgs, labels

In [11]:
def build_test_image_set(j):
    # Read image
    img = np.array(Image.open(os.path.join(test_path,test['fname'].iloc[j])))
        
    # Get dimensions
    dim_x = img.shape[0]
    dim_y = img.shape[1]
    
    # Set temp img
    tmp_img = ndimage.convolve(img.astype(np.float32), hpf, mode='mirror')
    tmp_img = img.astype(np.float32) - tmp_img
    
    # Find out if image needs padding using %
    if (dim_x % img_rows) != 0:
        tmp_img = np.pad(tmp_img, ([0,img_rows-(dim_x % img_rows)],[0,0],[0,0]), 'reflect')
    if (dim_y % img_cols) != 0:
        tmp_img = np.pad(tmp_img, ([0,0],[0,img_cols-(dim_y % img_cols)],[0,0]), 'reflect')
        
    tmp_img = tmp_img.astype(np.float16)
    
    # Next, cut up the little squares
#     cropped_imgs = []
#     for m in range(math.ceil(dim_x / img_rows)):
#         for n in range(math.ceil(dim_y / img_cols)):
#             xx_img = tmp_img[m*img_rows:(m+1)*img_rows,n*img_cols:(n+1)*img_cols,:]
#             cropped_imgs += [xx_img]
    
    return tmp_img

In [12]:
def build_flickr_image_set(j):
    # Read image
    img = np.array(Image.open(os.path.join(os.path.join(flickr_path,flickr['camera'].iloc[j]),flickr['fname'].iloc[j])), dtype=np.uint8)
    if len(img.shape)<3:
#         print('Error loading file, possible grayscale image',flickr['camera'].iloc[j], flickr['fname'].iloc[j])
        return 0,0
    if img.shape[2] != 3:
#         print('Error loading file, incorrect number of channels',flickr['camera'].iloc[j], flickr['fname'].iloc[j])
        return 0,0
    
    # Get dimensions
    dim_x = img.shape[0]
    dim_y = img.shape[1]
    
    # Set temp img
    tmp_img = ndimage.convolve(img.astype(np.float32), hpf, mode='mirror')
    tmp_img = img.astype(np.float32) - tmp_img
    tmp_img = tmp_img.astype(np.float16)
    
    try:
        cropped_imgs = []
        labels = []
        # get 5 images
        for i in range(8):
            m = np.random.randint(math.floor(dim_x / img_rows))
            n = np.random.randint(math.floor(dim_y / img_cols))
            xx_img = tmp_img[m*img_rows:(m+1)*img_rows,n*img_cols:(n+1)*img_cols,:]
            lab = np.zeros(len(cameras))
            lab[cameras.index(flickr['camera'].iloc[j])] = 1
            cropped_imgs += [xx_img]
            labels += [lab]
        return cropped_imgs, labels
    except:
#         print('Error loading file',flickr['camera'].iloc[j], flickr['fname'].iloc[j])
        return 0,0
    #xx_img = xx_img - ndimage.convolve(xx_img, hpf)

## StratifiedKFold Train split (0.9 Train, 0.1 Validation)
By splitting here, we can ensure that the cropped images aren't in both the training and validation sets.

In [13]:
from sklearn.model_selection import StratifiedKFold
n_folds = 10
skf = StratifiedKFold(n_folds, shuffle=True, random_state=np.random)
for train_index, test_index in skf.split(np.arange(len(train)), train['camera']):
    break

## Multiprocess train/val/test sets by index

In [14]:
with Pool(cpu_count()) as pool:
    print('Importing Flickr Images')
    flickr_imgs, flickr_labels = zip(*pool.map(build_flickr_image_set, range(len(flickr))))
    flickr_imgs = [x for x in flickr_imgs if not isinstance(x, int)]
    flickr_labels = [x for x in flickr_labels if not isinstance(x, int)]
    flickr_imgs = list(itertools.chain(*flickr_imgs))
    flickr_labels = list(itertools.chain(*flickr_labels))
    print('Done Importing Flickr Images')
    print('Importing Training Set')
    train_imgs, train_labels = zip(*pool.map(build_train_image_set, train_index))
    train_imgs = list(itertools.chain(*train_imgs))
    train_labels = list(itertools.chain(*train_labels))
    print('Done Importing Training Set')
    print('Importing Validation Set')
    val_imgs, val_labels = zip(*pool.map(build_train_image_set, test_index))
    val_imgs = list(itertools.chain(*val_imgs))
    val_labels = list(itertools.chain(*val_labels))
    print('Done Importing Validation Set')
    print('Importing Test Set')
    test_imgs = pool.map(build_test_image_set, range(len(test)))
#     test_imgs = list(itertools.chain(*test_imgs))
    print('Done Importing Test Set')

Importing Flickr Images


/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 7. 
  warnings.warn(str(msg))


Done Importing Flickr Images
Importing Training Set
Done Importing Training Set
Importing Validation Set
Done Importing Validation Set
Importing Test Set
Done Importing Test Set


## Build a SEInceptionResnetV2 Model

In [22]:
K.clear_session()

seres_model = DenseNet201(include_top=False, weights='imagenet', pooling=None, input_shape=(img_rows,img_cols,3))
# x = Flatten()(seres_model.output)
# x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(seres_model.output)
x = Dense(len(cameras), activation='softmax')(x)
smodel = Model(seres_model.input, x)

# MultiGPU Code
smodel.summary()
if gdev_count > 1:
    model = multi_gpu_model(smodel, gdev_count)
    model.summary()
else:
    model = smodel

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (

## Train Model

In [16]:
weights = class_weight.compute_class_weight('balanced',
                                            np.arange(len(cameras)),
                                            np.asarray(flickr_labels+train_labels).argsort(axis=1)[:,::-1][:,0])

In [23]:
def gen(x, y, b):
    while True:
        x_out, y_out = zip(*sample(list(zip(x, y)), b))
        yield np.asarray(x_out), np.asarray(y_out)

In [24]:
model_checkpoint = ModelCheckpoint('ieee-camera-'+session_name+'_acc.hdf5',
                                   monitor='val_acc', save_best_only=True, save_weights_only=True)
model_checkpoint2 = ModelCheckpoint('ieee-camera-'+session_name+'_loss.hdf5',
                                    monitor='val_loss', save_best_only=True, save_weights_only=True)
model_reducelr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10)
model_earlystop = EarlyStopping(patience=64, monitor='val_acc')
model.compile(optimizer=Adam(amsgrad=True), loss='categorical_crossentropy', metrics=['accuracy'])
batch_size = 8*gdev_count
train_history = model.fit_generator(gen(flickr_imgs+train_imgs, flickr_labels+train_labels, batch_size),
                                    steps_per_epoch=math.ceil(4096/batch_size), epochs=2**12,
                                    validation_data=gen(val_imgs, val_labels, batch_size),
                                    validation_steps=math.ceil(1024/batch_size),
                                    verbose=2, callbacks=[model_checkpoint, model_earlystop, model_checkpoint2, model_reducelr])

Epoch 1/4096
 - 205s - loss: 2.2108 - acc: 0.2195 - val_loss: 9.0120 - val_acc: 0.2041
Epoch 2/4096
 - 113s - loss: 1.9228 - acc: 0.3252 - val_loss: 3.7991 - val_acc: 0.1885
Epoch 3/4096
 - 114s - loss: 1.7108 - acc: 0.4109 - val_loss: 2.8209 - val_acc: 0.3057
Epoch 4/4096
 - 112s - loss: 1.5379 - acc: 0.4797 - val_loss: 4.9945 - val_acc: 0.2256
Epoch 5/4096
 - 115s - loss: 1.4690 - acc: 0.5020 - val_loss: 1.9225 - val_acc: 0.4307
Epoch 6/4096
 - 111s - loss: 1.3368 - acc: 0.5554 - val_loss: 3.8479 - val_acc: 0.2686
Epoch 7/4096
 - 113s - loss: 1.2588 - acc: 0.5806 - val_loss: 2.2740 - val_acc: 0.4541
Epoch 8/4096
 - 110s - loss: 1.1667 - acc: 0.6018 - val_loss: 3.6439 - val_acc: 0.2793
Epoch 9/4096
 - 115s - loss: 1.1127 - acc: 0.6240 - val_loss: 1.6943 - val_acc: 0.5352
Epoch 10/4096
 - 111s - loss: 1.0736 - acc: 0.6399 - val_loss: 2.7661 - val_acc: 0.2891
Epoch 11/4096
 - 110s - loss: 1.0584 - acc: 0.6414 - val_loss: 2.1840 - val_acc: 0.4580
Epoch 12/4096
 - 110s - loss: 0.9943 - ac

Epoch 95/4096
 - 111s - loss: 0.4228 - acc: 0.8469 - val_loss: 1.3391 - val_acc: 0.6299
Epoch 96/4096
 - 110s - loss: 0.3943 - acc: 0.8550 - val_loss: 0.6739 - val_acc: 0.7979
Epoch 97/4096
 - 110s - loss: 0.4013 - acc: 0.8623 - val_loss: 0.8569 - val_acc: 0.7393
Epoch 98/4096
 - 110s - loss: 0.4271 - acc: 0.8518 - val_loss: 0.6675 - val_acc: 0.8252
Epoch 99/4096
 - 110s - loss: 0.3888 - acc: 0.8645 - val_loss: 0.9236 - val_acc: 0.7334
Epoch 100/4096
 - 110s - loss: 0.4119 - acc: 0.8511 - val_loss: 0.7992 - val_acc: 0.7949
Epoch 101/4096
 - 110s - loss: 0.3913 - acc: 0.8613 - val_loss: 0.5782 - val_acc: 0.7939
Epoch 102/4096
 - 110s - loss: 0.3972 - acc: 0.8584 - val_loss: 0.7060 - val_acc: 0.8047
Epoch 103/4096
 - 111s - loss: 0.3836 - acc: 0.8596 - val_loss: 1.1568 - val_acc: 0.6562
Epoch 104/4096
 - 110s - loss: 0.3734 - acc: 0.8643 - val_loss: 0.6565 - val_acc: 0.8184
Epoch 105/4096
 - 112s - loss: 0.3576 - acc: 0.8738 - val_loss: 0.5552 - val_acc: 0.8271
Epoch 106/4096
 - 111s - l

 - 124s - loss: 0.1659 - acc: 0.9329 - val_loss: 0.1504 - val_acc: 0.9404
Epoch 188/4096
 - 124s - loss: 0.1980 - acc: 0.9287 - val_loss: 0.1557 - val_acc: 0.9424
Epoch 189/4096
 - 124s - loss: 0.1679 - acc: 0.9336 - val_loss: 0.1693 - val_acc: 0.9414
Epoch 190/4096
 - 124s - loss: 0.1694 - acc: 0.9351 - val_loss: 0.1793 - val_acc: 0.9346
Epoch 191/4096
 - 124s - loss: 0.1742 - acc: 0.9346 - val_loss: 0.1588 - val_acc: 0.9531
Epoch 192/4096
 - 124s - loss: 0.1722 - acc: 0.9409 - val_loss: 0.1931 - val_acc: 0.9434
Epoch 193/4096
 - 123s - loss: 0.1771 - acc: 0.9373 - val_loss: 0.1675 - val_acc: 0.9434
Epoch 194/4096
 - 124s - loss: 0.1743 - acc: 0.9370 - val_loss: 0.1634 - val_acc: 0.9375
Epoch 195/4096
 - 124s - loss: 0.1895 - acc: 0.9233 - val_loss: 0.1581 - val_acc: 0.9434
Epoch 196/4096
 - 124s - loss: 0.1894 - acc: 0.9343 - val_loss: 0.1614 - val_acc: 0.9561
Epoch 197/4096
 - 124s - loss: 0.2059 - acc: 0.9209 - val_loss: 0.1789 - val_acc: 0.9385
Epoch 198/4096
 - 123s - loss: 0.170

## Predict

In [25]:
# acc ver.
model.load_weights('ieee-camera-'+session_name+'_acc.hdf5')
predicted = model.predict(np.asarray(test_imgs), batch_size=32*gdev_count, verbose=1)
label_list = []
for p in predicted:
    label_list.append(cameras[p.argsort()[::-1][0]])
submit_df = pd.concat([test,pd.DataFrame({'camera':label_list})],axis=1)
submit_df.to_csv(session_name+'_acc.csv', index=False)
submit_df.head()

2640/2640 [==============================] - 54s 21ms/step


,fname,camera
0,img_0002a04_manip.tif,LG-Nexus-5x
1,img_001e31c_unalt.tif,iPhone-4s
2,img_00275cf_manip.tif,iPhone-6
3,img_0034113_unalt.tif,Samsung-Galaxy-S4
4,img_00344b7_unalt.tif,Motorola-X


In [26]:
# loss ver.
model.load_weights('ieee-camera-'+session_name+'_loss.hdf5')
predicted = model.predict(np.asarray(test_imgs), batch_size=32*gdev_count, verbose=1)
label_list = []
for p in predicted:
    label_list.append(cameras[p.argsort()[::-1][0]])
submit_df = pd.concat([test,pd.DataFrame({'camera':label_list})],axis=1)
submit_df.to_csv(session_name+'_loss.csv', index=False)
submit_df.head()

2640/2640 [==============================] - 37s 14ms/step


,fname,camera
0,img_0002a04_manip.tif,LG-Nexus-5x
1,img_001e31c_unalt.tif,iPhone-4s
2,img_00275cf_manip.tif,iPhone-6
3,img_0034113_unalt.tif,Samsung-Galaxy-S4
4,img_00344b7_unalt.tif,Motorola-X


## Shutdown Notebook Kernel on Finish

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>